In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler, StandardScaler
from category_encoders import CountEncoder
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score

In [2]:
train_df = pd.read_csv('train.csv', na_values = '?')
train_df.isnull().sum()

age                  0
workclass         1268
fnlwgt               0
education            0
education.num        0
marital.status       0
occupation        1273
relationship         0
race                 0
sex                  0
capital.gain         0
capital.loss         0
hours.per.week       0
native.country     427
income               0
dtype: int64

In [3]:
test_df = pd.read_csv('test.csv', na_values = '?')
test_df.isnull().sum()
test_df.shape

(9769, 15)

In [4]:
train_df['capital'] = train_df['capital.gain'] - train_df['capital.loss']
test_df['capital'] = test_df['capital.gain'] - test_df['capital.loss']

In [5]:
train_df.head(5)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income,capital
0,58,Private,290661,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,0
1,62,Private,109463,Some-college,10,Separated,Sales,Unmarried,White,Female,0,1617,33,United-States,<=50K,-1617
2,33,Private,137088,Bachelors,13,Married-civ-spouse,Machine-op-inspct,Husband,Other,Male,0,0,40,Ecuador,<=50K,0
3,24,Private,117767,Assoc-acdm,12,Never-married,Sales,Own-child,White,Male,0,0,20,United-States,<=50K,0
4,67,Self-emp-not-inc,431426,HS-grad,9,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,2,United-States,<=50K,0


In [6]:
train_df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country',
       'income', 'capital'],
      dtype='object')

In [7]:
feature_columns = ['age', 'workclass', 'fnlwgt', 'education', 'education.num',
       'marital.status', 'occupation', 'relationship', 'race', 'sex',
       'capital.gain', 'capital.loss', 'hours.per.week', 'native.country', 'capital']
label_columns = 'income'

In [8]:
X_train = train_df.loc[:, feature_columns].values
y_train = train_df.loc[:, label_columns].values

X_test = test_df.loc[:, feature_columns].values
y_test = test_df.loc[:, label_columns].values

## Missing Value

In [9]:
def MissingValue(X, y):
    workclass_imputer = SimpleImputer(np.nan, strategy = 'most_frequent')
    X[:, 1] = workclass_imputer.fit_transform(X[:, 1].reshape(-1, 1)).ravel()
    
    occupation_imputer = SimpleImputer(np.nan, strategy = 'most_frequent')
    X[:, 6] = occupation_imputer.fit_transform(X[:, 1].reshape(-1, 1)).ravel()
    
    country_imputer = SimpleImputer(np.nan, strategy = 'most_frequent')
    X[:, 13] = country_imputer.fit_transform(X[:, 13].reshape(-1, 1)).ravel()
    return X, y



In [10]:
X_train, y_train = MissingValue(X_train, y_train)
X_test, y_test = MissingValue(X_test, y_test)

In [11]:
train_dataset = pd.DataFrame.from_records(X_train)
train_dataset.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

In [12]:
test_dataset = pd.DataFrame.from_records(X_test)
test_dataset.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

## Chuẩn hóa dữ liệu

In [13]:
train_df.head()

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income,capital
0,58,Private,290661,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,0
1,62,Private,109463,Some-college,10,Separated,Sales,Unmarried,White,Female,0,1617,33,United-States,<=50K,-1617
2,33,Private,137088,Bachelors,13,Married-civ-spouse,Machine-op-inspct,Husband,Other,Male,0,0,40,Ecuador,<=50K,0
3,24,Private,117767,Assoc-acdm,12,Never-married,Sales,Own-child,White,Male,0,0,20,United-States,<=50K,0
4,67,Self-emp-not-inc,431426,HS-grad,9,Married-civ-spouse,Adm-clerical,Wife,White,Female,0,0,2,United-States,<=50K,0


In [14]:
def Normalize(X, y):
    label_encoder = LabelEncoder()
    one_hot_encoder = OneHotEncoder()
    min_max_encoder = MinMaxScaler()
    standard_encoder = StandardScaler()
    count_encoder = CountEncoder()
    X[:, 0] = min_max_encoder.fit_transform(X[:, 0].reshape(-1, 1)).ravel() #age

    X[:, 1] = label_encoder.fit_transform(X[:, 1].reshape(-1, 1)).ravel() #workclass
    
    X[:, 2] = min_max_encoder.fit_transform(X[:, 2].reshape(-1, 1)).ravel() #fnlwgt
    
#     X[:, 3] = label_encoder.fit_transform(X[:, 3].reshape(-1, 1)).ravel()
    X[:, 3] = count_encoder.fit_transform(X[:, 3].reshape(-1, 1)).values.ravel() #education
    X[:, 3] = min_max_encoder.fit_transform(X[:, 3].reshape(-1, 1)).ravel()
    
    X[:, 5] = label_encoder.fit_transform(X[:, 5].reshape(-1, 1)).ravel() #marital.status
    
#     X[:, 6] = label_encoder.fit_transform(X[:, 6].reshape(-1, 1)).ravel() #occupation
    X[:, 6] = count_encoder.fit_transform(X[:, 6].reshape(-1, 1)).values.ravel()
    X[:, 6] = min_max_encoder.fit_transform(X[:, 6].reshape(-1, 1)).ravel()
    
    
    X[:, 7] = label_encoder.fit_transform(X[:, 6].reshape(-1, 1)).ravel() #relationship
    
    X[:, 8] = label_encoder.fit_transform(X[:, 8].reshape(-1, 1)).ravel() #race
    
    X[:, 9] = label_encoder.fit_transform(X[:, 8].reshape(-1, 1)).ravel() #sex
#     X[:, 10] = min_max_encoder.fit_transform(X[:, 10].reshape(-1, 1)).ravel() #capital.gain
#     X[:, 11] = min_max_encoder.fit_transform(X[:, 11].reshape(-1, 1)).ravel() #capital.loss

    X[:, 12] = min_max_encoder.fit_transform(X[:, 12].reshape(-1, 1)).ravel() #hours.per.week
    
    X[:, 13] = count_encoder.fit_transform(X[:, 13].reshape(-1, 1)).values.ravel() #native.country
    X[:, 13] = min_max_encoder.fit_transform(X[:, 13].reshape(-1, 1)).ravel()
    
    X[:, 14] = min_max_encoder.fit_transform(X[:, 14].reshape(-1, 1)).ravel()
    
#     race_one_hot = one_hot_encoder.fit_transform(X[:, 1].reshape(-1, 1)).toarray()
#     X = np.delete(X, 1, axis = 1)
#     X = np.concatenate((X[:, 0:1], race_one_hot, X[:, 1:]), axis = 1)
    
    X = np.delete(X, 11, axis = 1)
    X = np.delete(X, 10, axis = 1)
    y = label_encoder.fit_transform(y) #income
    return X, y

In [15]:
X_train, y_train = Normalize(X_train, y_train)
X_test, y_test = Normalize(X_test, y_test)

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(22792, 13) (22792,) (9769, 13) (9769,)


D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_

In [16]:
clf = GaussianNB()
# clf = MultinomialNB()
# clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [17]:
print("Accuracy score:", accuracy_score(y_test, y_pred))

Accuracy score: 0.8158460436073293


## Implement

In [28]:
def seperate_classes(X, y):
    unique_classes = np.unique(y)
    seperated_classes = {unique_class:[] for unique_class in unique_classes}
    for unique_class in unique_classes:
        seperated_classes[unique_class].append(X[np.where(y==unique_class)[0]])
    return seperated_classes

def calculate_label_probs(seperated_classes):
    n_samples = sum(len(value[0] for value in seperated_classes.values()))
    label_probs = {seperated_class: len(seperated_classes[seperated_class][0])/float(n_samples) for seperated_class in seperated_classes}
    return label_probs

def gaussian(xi, mean_xi, std_i):
    return np.exp(-np.power(xi-mean_xi, 2.)/(2*np.power(std_i, 2.)))/(np.sqrt(2*pi*np.power(std_i, 2.)))

In [29]:
global std, mean, label_probs
def fit(X_train, y_train):
    global std
    std = {}
    global mean
    mean = {}
    global label_probs
    label_probs = {}
    seperated_classes = seperate_classes(X_train, y_train)
    label_probs = calculate_label_probs(seperated_classes)
    for seperated_class in seperated_classes:
        std[seperated_class] = np.std(seperated_classes[seperated_class], axis = 1).flatten()
        mean[seperated_class] = np.mean(seperated_classes[seperated_class], axis = 1).flatten()

In [30]:
def predict(X_test):
    global std
    global mean
    global label_prods
    assert std or mean or label_prods, "Fit before predicting"
    result = []
    for x_row in X_test:
        conditional_probs = []
        for label in label_prods:
            mul = 1
            for index, feature in enumerate(x_row):
                gauss = gaussian(feature, mean[label][index], std[label][index])
                mul = mul *gauss
            conditional_prob = label_prods[label]*mul
            conditional_probs.append(conditional_prob)
        result.append(np.argmax(conditional_probs))
    return result
